In [1]:
import nest_asyncio
nest_asyncio.apply()

import aiohttp
import asyncio
import json
import pandas as pd
from datetime import datetime 
from datetime import timedelta


ums_base_url = "https://app.edoofa.com/version-test/api/1.1/obj"
ums_bearer_token = "786720e8eb68de7054d1149b56cc04f9"
ums_headers = {'Authorization': f'Bearer {ums_bearer_token}'}


async def fetch_table_data(session, base_url, headers, table, constraints=None):
    records = []
    cursor = 0
    total_fetched = 0

    while True:
        params = {'limit': 100, 'cursor': cursor}
        if constraints:
            params['constraints'] = json.dumps(constraints)

        api_url = f"{base_url}/{table}"
        print(f"Fetching {table} data from {base_url}... Cursor: {cursor}")

        async with session.get(api_url, headers=headers, params=params) as response:
            if response.status != 200:
                print(f"Failed to fetch data from {table}: {await response.text()}")
                break

            data = await response.json()
            new_records = data['response']['results']
            records.extend(new_records)
            total_fetched += len(new_records)

            print(f"Fetched {len(new_records)} new records, Total fetched: {total_fetched}")

            cursor += 100

            if len(new_records) < 100:
                print(f"Exiting loop, fetched less than 100 records.")
                break

    df = pd.DataFrame(records)
    print(f"Fetched {len(df)} records for {table}.")
    return df


def get_percentage(number_of_present, total_sessions):
    if total_sessions == 0:
        return 0  
    percent = (number_of_present / total_sessions) * 100
    return percent


def process_attendance_for_student(group, latest_summary_date, common_earliest_date):
    # Initialize a list to store monthly summary data
    monthly_summaries = []
    
    # Iterate through each month from common_earliest_date to latest_attendance_date
    current_date = common_earliest_date
    while current_date <= latest_summary_date:
        # Filter the group for the current month
        month_attendance = group[(group['date'] >= current_date) & (group['date'] < current_date + pd.offsets.MonthEnd(1))]
        
        # Calculate the monthly attendance percentage
        number_of_present = month_attendance['present'].sum()
        total_sessions = len(month_attendance)
        monthly_percentage = get_percentage(number_of_present, total_sessions)
        
        # Calculate the avg-att-percent-till-last-month
        # Filter the group for dates less than the first day of the current month
        previous_attendance = group[group['date'] < current_date]
        total_present_till_last_month = previous_attendance['present'].sum()
        total_sessions_till_last_month = len(previous_attendance)
        avg_percentage_till_last_month = get_percentage(total_present_till_last_month, total_sessions_till_last_month)
        
        # Add the calculated data to monthly_summaries
        monthly_summaries.append({
            'admissions-group-name': group['admissions-group-name'].iloc[0],
            'ewyl-group-name': group['ewyl-group-name'].iloc[0],
            'first-day-of-month': current_date,
            'attendance-percentage': monthly_percentage,
            'avg-att-percent-till-last-month': avg_percentage_till_last_month,
            'month': current_date.month,
            'year': current_date.year
        })
        
        # Move to the next month
        current_date += pd.offsets.MonthBegin(1)
    
    # Convert monthly_summaries to DataFrame and return
    return pd.DataFrame(monthly_summaries)

async def main():
    async with aiohttp.ClientSession() as session:
        ums_att_summary_df = await fetch_table_data(session, ums_base_url, ums_headers, "Attendance-Summary")
        ums_att_df = await fetch_table_data(session, ums_base_url, ums_headers, "Attendance")

        # Convert 'date' columns to datetime
        ums_att_df['date'] = pd.to_datetime(ums_att_df['date'])
        ums_att_summary_df['first-day-of-month'] = pd.to_datetime(ums_att_summary_df['first-day-of-month'])

        # Find the common earliest attendance date
        common_earliest_date = ums_att_df['date'].min()

        # Group the attendance data by admissions-group-name
        grouped_attendance = ums_att_df.groupby('admissions-group-name')

        for admissions_group_name, group in grouped_attendance:
            # Sort the attendance records by date in descending order
            sorted_group = group.sort_values(by='date', ascending=False)

            # Get the latest attendance date for the current student/group
            latest_attendance_date = sorted_group['date'].iloc[0]

            # Find the greatest 'first-day-of-month' for the current student in the summary table
            student_summary = ums_att_summary_df[ums_att_summary_df['admissions-group-name'] == admissions_group_name]
            if not student_summary.empty:
                latest_summary_date = student_summary['first-day-of-month'].max()
            else:
                # If there's no summary data for the student, use the common earliest date
                latest_summary_date = common_earliest_date

            # Check if the latest attendance date is greater than the latest summary date
            if latest_attendance_date > latest_summary_date:
                # Function to process attendance and calculate summaries for the months until the condition is met
                process_attendance_for_student(group, latest_summary_date, common_earliest_date)
                # This function would calculate the attendance percentage for each month from the common earliest date
                # to the latest attendance date, and for months without data, it would carry on the calculations with 0% attendance

        # After processing all students/groups, you might need to update the Attendance-Summary table with new or updated summaries

await main()



Fetching Attendance-Summary data from https://app.edoofa.com/version-test/api/1.1/obj... Cursor: 0
Fetched 69 new records, Total fetched: 69
Exiting loop, fetched less than 100 records.
Fetched 69 records for Attendance-Summary.
Fetching Attendance data from https://app.edoofa.com/version-test/api/1.1/obj... Cursor: 0
Fetched 100 new records, Total fetched: 100
Fetching Attendance data from https://app.edoofa.com/version-test/api/1.1/obj... Cursor: 100
Fetched 100 new records, Total fetched: 200
Fetching Attendance data from https://app.edoofa.com/version-test/api/1.1/obj... Cursor: 200
Fetched 100 new records, Total fetched: 300
Fetching Attendance data from https://app.edoofa.com/version-test/api/1.1/obj... Cursor: 300
Fetched 100 new records, Total fetched: 400
Fetching Attendance data from https://app.edoofa.com/version-test/api/1.1/obj... Cursor: 400
Fetched 100 new records, Total fetched: 500
Fetching Attendance data from https://app.edoofa.com/version-test/api/1.1/obj... Cursor: